# Knowledge Retrieval

OpenAIが提供するAssistantAPIのうちの一つの機能．モデル外部から知識を取得し回答する．

- ドキュメント:https://platform.openai.com/docs/assistants/tools/knowledge-retrieval

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/nlp/llm_framework/knowledge_retrieval.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai

In [1]:
from openai import OpenAI
import os

client = OpenAI()

#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"


### 入力設定

In [2]:
instruction = "あなたはData Science Wikiに詳しい専門家です．ナレッジを活用して正確な回答をしましょう．"
model = "gpt-4-1106-preview"
content = "Data Science Wikiとは何ですか？"


In [3]:
file = client.files.create(
  file=open("../../README.md", "rb"),
  purpose='assistants'
)

# Add the file to the assistant
assistant = client.beta.assistants.create(
  instructions = instruction,
  model = model,
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

# スレッドの作成
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = content,
    #file_ids=[file.id]
)


In [4]:
# スレッドにメッセージが追加されていることを確認する
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)


user : Data Science Wikiとは何ですか？


### Knowledge Retrievalの実行

In [5]:
# アシスタントにリクエスト
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
    )
print(run.status)


queued


In [6]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)


user : Data Science Wikiとは何ですか？
assistant : 


In [7]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)
run_steps


SyncCursorPage[RunStep](data=[RunStep(id='step_eu0HXz0NtxZCsEV1lceE0qcf', assistant_id='asst_0QfM9OUEQJMvEKrY5o5XBMt0', cancelled_at=None, completed_at=None, created_at=1699543843, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_6CvgcBkjyuCILJmPA3H7xG8z', status='in_progress', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_kB62HkiixnPRQJJrgkerTuIk'), type='message_creation'), thread_id='thread_T3QNKzjng9vDxX2C1yk5PyuK', type='message_creation', expires_at=1699544442)], object='list', first_id='step_eu0HXz0NtxZCsEV1lceE0qcf', last_id='step_eu0HXz0NtxZCsEV1lceE0qcf', has_more=False)